In [11]:
import pandas as pd
import weaviate
import weaviate.classes as wvc
import os
import requests
import json
from pprint import pprint
import tqdm
import matplotlib
from main.keys import open_ai_key, weaviate_url, weaviate_key
import matplotlib.pyplot as plt
import numpy as np

In [10]:
data_ner = pd.read_feather("job_postings_with_entities_updated.fth")
# data_original = pd.read_feather("data/postings_w_embeddings_v2_original.fth")
print(data_ner.shape)
# print(data_original.shape)

(1100000, 50)


In [3]:
# data_wv_uuid = data_ner[['wv_uuid']]

In [4]:
data_wv_uuid.to_feather('data/data_wv_uuid.fth')

In [4]:
data_ner = data_ner.drop(columns=[ 'scraped','formatted_work_type',
       'location', 'applies', 'original_listed_time', 'remote_allowed',
       'application_type', 'expiry', 'inferred_benefits', 'skills_desc', 'views', 'job_region', 'listed_time', 'degree', 'state', 'country_company',
       'city', 'zip_code', 'address',
       'posting_domain','description_company', 'sponsored','closed_time','job_posting_url','application_url','url', 'description', 'text'])

In [5]:
data_ner.to_feather("data/postings_w_embeddings_v2_small.fth")

In [6]:
data_ner.columns

Index(['job_id', 'company_id', 'work_type', 'formatted_experience_level',
       'years_experience', 'title', 'country', 'country_code', 'job_functions',
       'industry_names', 'company_name', 'company_size', 'text',
       'entities_COMPANY', 'entities_METHODS', 'entities_TOOLS',
       'entities_EXPERIENCE', 'entities_LEVEL', 'entities_REMOTE',
       'entities_RESPONSABILITY', 'entities_TITLE', 'entities_QUALIFICATION',
       'vector', 'wv_uuid', 'annotations'],
      dtype='object')

In [33]:
data_ner.head()
data_ner.memory_usage(deep=True)

Index                            8800000
job_id                           8800000
scraped                          8800000
company_id                       8800000
work_type                       72494590
formatted_work_type             72494590
location                        78878323
job_posting_url                138600000
applies                          8800000
original_listed_time             8800000
remote_allowed                   8800000
application_url                294844909
application_type                76268070
expiry                           8800000
inferred_benefits                8800000
closed_time                      8800000
formatted_experience_level      73548240
years_experience                 8800000
description                   6453577742
title                          104624620
skills_desc                     28860943
views                            8800000
job_region                       8800000
listed_time                      8800000
degree          

In [12]:
data = data_ner.copy()
data['entities_TOOLS'] = data['entities_TOOLS'].apply(lambda x: x.tolist())
data['entities_METHODS'] = data['entities_ METHODS'].apply(lambda x: x.tolist())
data['entities_REMOTE'] = data['entities_REMOTE'].apply(lambda x: x.tolist())
data['entities_EXPERIENCE'] = data['entities_EXPERIENCE'].apply(lambda x: x.tolist())
data['entities_RESPONSABILITY'] = data['entities_RESPONSABILITY'].apply(lambda x: x.tolist())
data['job_functions'] = data['job_functions'].apply(lambda x: eval(x))
data['industry_names'] = data['industry_names'].apply(lambda x: eval(x))
data_ner = data.copy()

In [5]:
data = data_original.copy()
data['entities_TOOLS'] = data['entities_TOOLS'].apply(lambda x: x.tolist())
data['entities_METHODS'] = data['entities_METHODS'].apply(lambda x: x.tolist())
data['entities_REMOTE'] = data['entities_REMOTE'].apply(lambda x: x.tolist())
data['entities_EXPERIENCE'] = data['entities_EXPERIENCE'].apply(lambda x: x.tolist())
data['entities_RESPONSABILITY'] = data['entities_RESPONSABILITY'].apply(lambda x: x.tolist())
data['job_functions'] = data['job_functions'].apply(lambda x: x.tolist())
data['industry_names'] = data['industry_names'].apply(lambda x: x.tolist())
data_original = data.copy()

In [13]:
entities = data_ner[['company_name','title','formatted_experience_level','job_functions', 'industry_names', 'entities_TOOLS', 'entities_METHODS', 'entities_REMOTE', 'entities_EXPERIENCE', 'entities_RESPONSABILITY',]]

In [14]:
entities_dict = {}

for col in entities.columns:
    col_list = entities[col].tolist()
    print(col)
    
    col_list_flat = []
    for item in col_list:
        if isinstance(item, (list, tuple)):
            col_list_flat.extend([subitem for subitem in item if subitem is not None])
        elif item is not None:
            col_list_flat.append(item)
    
    entities_dict[col] = col_list_flat

company_name
title
formatted_experience_level
job_functions
industry_names
entities_TOOLS
entities_METHODS
entities_REMOTE
entities_EXPERIENCE
entities_RESPONSABILITY


In [17]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def calculate_embedding_metrics(df_sample):
    if 'description' not in df_sample.columns:
        raise ValueError("DataFrame must contain 'description' column")

    ner_columns = [
        'company_name', 'title', 'formatted_experience_level', 'job_functions',
        'industry_names', 'entities_TOOLS', 'entities_METHODS', 'entities_REMOTE',
        'entities_EXPERIENCE', 'entities_RESPONSABILITY'
    ]

    total_description_words = df_sample['description'].apply(lambda x: len(str(x).split())).sum()

    total_ner_words = 0
    all_ners = []
    for col in ner_columns:
        if col in df_sample.columns:
            total_ner_words += df_sample[col].apply(lambda x: sum(len(str(entity).split()) for entity in (x or []) if entity is not None)).sum()
            all_ners.extend([str(entity) for ner_list in df_sample[col].dropna() for entity in (ner_list or []) if entity is not None])

    unique_ners = set(all_ners)
    unique_ner_words = sum(len(entity.split()) for entity in unique_ners)

    return {
        'full_description_words': total_description_words,
        'full_ner_words': total_ner_words,
        'unique_ner_words': unique_ner_words
    }

def generate_plot_data(df, sample_sizes):
    plot_data = []
    for size in sample_sizes:
        sample = df.sample(n=size, random_state=42)
        metrics = calculate_embedding_metrics(sample)
        metrics['sample_size'] = size
        plot_data.append(metrics)
    return pd.DataFrame(plot_data)

# Example usage:
# Assuming 'data_ner' is your full DataFrame
sample_sizes = [100, 500, 1000, 5000, 10000, 20000, 30000, 40000, 50000, 60000, 70000, 80000, 100_000, 200_000,300_000,400_000,500_000,750_000,1_000_000,int(len(data_ner))]  # Adjust as needed
plot_data = generate_plot_data(data_ner, sample_sizes)

# Create the Plotly figure
fig = make_subplots(rows=1, cols=1, shared_xaxes=True)

# Add traces for each metric
fig.add_trace(
    go.Scatter(x=plot_data['sample_size'], y=plot_data['full_description_words'],
               mode='lines+markers', name='Full Description', line=dict(color='blue'))
)
fig.add_trace(
    go.Scatter(x=plot_data['sample_size'], y=plot_data['full_ner_words'],
               mode='lines+markers', name='Full NERs', line=dict(color='red'))
)
fig.add_trace(
    go.Scatter(x=plot_data['sample_size'], y=plot_data['unique_ner_words'],
               mode='lines+markers', name='Unique NERs', line=dict(color='green'))
)

# Update layout
fig.update_layout(
    title='Number of Words vs Listings Embedded',
    xaxis_title='Number of Listings',
    yaxis_title='Number of Words',
    legend_title='Embedding Method',
    hovermode='x unified',
    template='plotly_white'
)

# Add gridlines
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')

# Show the plot
fig.show()


In [9]:
import pandas as pd

def generate_unique_ner_dict(df):
    # List of NER columns
    ner_columns = [
        'company_name', 'title', 'formatted_experience_level', 'job_functions', 
        'industry_names', 'entities_TOOLS', 'entities_METHODS', 'entities_REMOTE', 
        'entities_EXPERIENCE', 'entities_RESPONSABILITY'
    ]

    unique_ner_dict = {}

    for col in ner_columns:
        if col in df.columns:
            # Flatten the list of lists and get unique entries
            unique_entries = set()
            for entry_list in df[col].dropna():
                if isinstance(entry_list, list):
                    unique_entries.update(str(item) for item in entry_list if item is not None)
                else:
                    unique_entries.add(str(entry_list))
            
            # Store the unique entries in the dictionary
            unique_ner_dict[col] = sorted(list(unique_entries))

    return unique_ner_dict

# Example usage:
# Assuming 'data_ner' is your DataFrame
unique_ners = generate_unique_ner_dict(data_ner)

# Print the results
for column, entries in unique_ners.items():
    print(f"\n{column}:")
    print(f"Number of unique entries: {len(entries)}")
    print("Sample entries (up to 10):", entries[:10])


company_name:
Number of unique entries: 13750
Sample entries (up to 10): [' Coeur d’Alene Casino Resort Hotel', ' Farmers® Insurance of Michigan - District Office ', ' Free Rein Life Ltd', ' Georgia Pacific', ' Health Matching Account Services', ' Pilot Company', ' The Times-Picayune | Nola.com', '"A" Best International Placement Services, LLC ', '#DegreesNYC', '#twiceasnice Recruiting']

title:
Number of unique entries: 62527
Sample entries (up to 10): ['  Lead Frontend Engineer ', ' Account Manager Entry Level', ' Associate Technical Account Manager', ' Corporate Lawyer - London - 1-5PQE', ' Falcon 2000Easy', ' Field QA Specialist', ' Field Service Engineer', ' Head Teller (PC 14)', ' Instructional Designer and Technologist I', ' Intune Endpoint Engineer']

formatted_experience_level:
Number of unique entries: 6
Sample entries (up to 10): ['Associate', 'Director', 'Entry level', 'Executive', 'Internship', 'Mid-Senior level']

job_functions:
Number of unique entries: 35
Sample entrie

In [11]:
from main.keys import open_ai_key, weaviate_url, weaviate_key
import os
import weaviate
import openai
from typing import List, Dict
import numpy as np
from tqdm import tqdm

os.environ["OPENAI_APIKEY"] = open_ai_key
os.environ["WCD_URL"] = weaviate_url
os.environ["WCD_API_KEY"] = weaviate_key

openai_api_key = os.environ.get("OPENAI_APIKEY", "<your OpenAI API key if not set as env var>")
openai.api_key = openai_api_key

def vectorize(texts: List[str]) -> List[List[float]]:
    # print(texts)
    response = openai.embeddings.create(
        input=texts, model="text-embedding-3-small"
    )
    # print(f"{[item.embedding for item in response.data]=}")
    return [item.embedding for item in response.data]

def prepare_entity_texts(entities_dict):
    prepared_texts = {}
    for column, entities in entities_dict.items():
        if column.startswith('entities_'):
            prefix = column.split('_')[1] + ': '
        else:
            prefix = column + ': '
        prepared_texts[column] = [prefix + entity for entity in entities]
    return prepared_texts

def generate_embeddings_for_entities(entities_dict: Dict[str, List[str]], batch_size: int = 1000) -> Dict[str, Dict[str, List[float]]]:
    embeddings_dict = {}
    prepared_texts = prepare_entity_texts(entities_dict)
    
    for entity_type, texts in prepared_texts.items():
        print(f"Generating embeddings for {entity_type}")
        # if entity_type != "entities_TOOLS":
            # continue
        print(f"Generating embeddings for {entity_type}")
        entity_embeddings = {}
        
        for i in tqdm(range(0, len(texts), batch_size)):
            batch = texts[i:i+batch_size]
            try:
                embeddings = vectorize(batch)
                for text, embedding in zip(batch, embeddings):
                    entity = text.split(': ', 1)[1]  # Extract the entity part
                    entity_embeddings[entity] = embedding
            except Exception as e:
                print(f"Error in batch {i}-{i+batch_size} for {entity_type}: {str(e)}")
            # break
        
        embeddings_dict[entity_type] = entity_embeddings
    
    return embeddings_dict

# Generate embeddings for all unique entities
embeddings_dict = generate_embeddings_for_entities(unique_ners)


# Save embeddings to disk
np.save('data/entity_embeddings_ner.npy', embeddings_dict)

print("Embeddings generation completed and saved to disk.")


Generating embeddings for company_name
Generating embeddings for company_name


100%|██████████| 14/14 [00:39<00:00,  2.82s/it]


Generating embeddings for title
Generating embeddings for title


100%|██████████| 63/63 [02:54<00:00,  2.77s/it]


Generating embeddings for formatted_experience_level
Generating embeddings for formatted_experience_level


...

Generating embeddings for entities_EXPERIENCE
Generating embeddings for entities_EXPERIENCE


100%|██████████| 42/42 [02:11<00:00,  3.13s/it]


Generating embeddings for entities_RESPONSABILITY
Generating embeddings for entities_RESPONSABILITY


100%|██████████| 279/279 [13:34<00:00,  2.92s/it]


Embeddings generation completed and saved to disk.


In [4]:
embeddings_dict = np.load('data/entity_embeddings_ner.npy', allow_pickle=True).item()

In [5]:
import pandas as pd
import numpy as np

flattened_data = [
    {
        'type': entity_type,
        'value': entity_value,
        'embedding': np.array(embedding)
    }
    for entity_type, entities in embeddings_dict.items()
    for entity_value, embedding in entities.items()
]

df_embeddings = pd.DataFrame(flattened_data)

print(df_embeddings.head())
print(df_embeddings.shape)

           type                                              value  \
0  company_name                  Coeur d’Alene Casino Resort Hotel   
1  company_name   Farmers® Insurance of Michigan - District Off...   
2  company_name                                 Free Rein Life Ltd   
3  company_name                                    Georgia Pacific   
4  company_name                   Health Matching Account Services   

                                           embedding  
0  [0.018065080046653748, -0.0579984188079834, 0....  
1  [0.0029444394167512655, 0.0003384020528756082,...  
2  [0.05414898321032524, -0.026501184329390526, 0...  
3  [-0.00013810588279739022, -0.01255276892334222...  
4  [-0.004248972982168198, -0.017296072095632553,...  
(422783, 3)


In [6]:
df_embeddings.type.value_counts()

type
entities_RESPONSABILITY       278540
title                          62527
entities_EXPERIENCE            41381
entities_METHODS               16646
company_name                   13750
entities_TOOLS                  8939
entities_REMOTE                  612
industry_names                   347
job_functions                     35
formatted_experience_level         6
Name: count, dtype: int64

In [7]:
df_embeddings.to_feather('sub_component_embeddings.fth')

In [6]:
df_embeddings_tools = df_embeddings.query('type == "entities_RESPONSABILITY"')
df_embeddings_tools.shape

(278540, 3)

In [29]:
import pandas as pd

# Assuming your DataFrame is named 'df'

# List of columns we want to include in our dictionary
columns_to_include = [
    'company_name', 'title', 'formatted_experience_level', 'job_functions', 
    'industry_names', 'entities_TOOLS', 'entities_METHODS', 'entities_REMOTE', 
    'entities_EXPERIENCE', 'entities_RESPONSABILITY'
]

# Convert DataFrame to dictionary
job_dict = {}

for _, row in data_ner.iterrows():
    job_id = row['job_id']
    job_dict[job_id] = {col: row[col] for col in columns_to_include}

    # Convert list-like strings to actual lists
    for col in columns_to_include:
        if isinstance(job_dict[job_id][col], str) and job_dict[job_id][col].startswith('[') and job_dict[job_id][col].endswith(']'):
            try:
                job_dict[job_id][col] = eval(job_dict[job_id][col])
            except:
                # If eval fails, keep the original string
                pass

# Print a sample of the dictionary to verify
sample_job_id = next(iter(job_dict))
sample_job_id = next(iter(job_dict))
print(f"Sample entry for job_id {sample_job_id}:")
print(job_dict[sample_job_id])

# Print the total number of jobs in the dictionary
print(f"\nTotal number of jobs: {len(job_dict)}")


Sample entry for job_id 3940522647:
{'company_name': 'AlliedTravelCareers', 'title': 'Travel PT - $2,410 per week in Plano, TX', 'formatted_experience_level': 'Mid-Senior level', 'job_functions': array(['Management', 'Manufacturing'], dtype=object), 'industry_names': array(['Hospitals and Health Care'], dtype=object), 'entities_TOOLS': array(['BluePipes.', 'BluePipes.'], dtype=object), 'entities_METHODS': array(['Partner of the Year, Largest Patient Impact, and Most Engaged Travel Associate.'],
      dtype=object), 'entities_REMOTE': array([], dtype=object), 'entities_EXPERIENCE': array([], dtype=object), 'entities_RESPONSABILITY': array([], dtype=object)}

Total number of jobs: 99513


In [17]:
data_original.query('job_id == 3940522647').description.values

array(["AlliedTravelCareers is working with FlexCare Medical Staffing to find a qualified PT in Plano, Texas, 75023! Pay Information $2,410 per week About The Position FlexCare is a nationwide leader in the staffing of travel nurses and clinicians. With access to thousands of facilities around the country, our mission is to deliver premier travel staffing solutions with integrity and transparency. FlexCare’s exceptional service and reputation make it one of the most awarded healthcare staffing companies in the industry each year, ranking as one of the top travel nursing companies in the nation from industry authorities like Highway Hypodermics, Travel Nursing Central, and BluePipes. FlexCare has also been recognized by hospital MSP's with awards such as; Partner of the Year, Largest Patient Impact, and Most Engaged Travel Associate. 8881465EXPPLAT Benefits FlexCare's robust benefits package for all travel assignments includes: MaxPay - our commitment to offer the maximum pay package wi

In [32]:
df_embeddings.type.value_counts()

type
entities_RESPONSABILITY       278540
title                          62527
entities_EXPERIENCE            41381
entities_METHODS               16646
company_name                   13750
entities_TOOLS                  8939
entities_REMOTE                  612
industry_names                   347
job_functions                     35
formatted_experience_level         6
Name: count, dtype: int64

In [40]:
import numpy as np
import pandas as pd

# Assuming you have your job dictionary as 'job_dict' and your embeddings dataframe as 'embeddings_df'

# First, create a dictionary for faster lookup
embeddings_lookup = {}
for _, row in df_embeddings.iterrows():
    if row['type'] not in embeddings_lookup:
        embeddings_lookup[row['type']] = {}
    embeddings_lookup[row['type']][row['value']] = row.name
# Transform the job dictionary
transformed_job_dict = {}

for job_id, job_data in job_dict.items():
    transformed_job_data = {}
    transformed_job_data['embeddings'] = {}
    transformed_job_data['embedding_indexes'] = []
    for key, value in job_data.items():
        if key in ['company_name', 'title', 'formatted_experience_level']:
            # For single string fields
            if key in embeddings_lookup and value in embeddings_lookup[key]:
                index = embeddings_lookup[key][value]
                transformed_job_data['embedding_indexes'].append(index)
                transformed_job_data['embeddings'][key] = {value: df_embeddings.loc[index, 'embedding']}

        elif key in ['job_functions', 'industry_names'] or key.startswith('entities_'):
            # For list fields including entity fields
            value = value.tolist()
            if isinstance(value, list):
                field_embeddings = {}
                for item in value:
                    if key in embeddings_lookup and item in embeddings_lookup[key]:
                        index = embeddings_lookup[key][item]
                        transformed_job_data['embedding_indexes'].append(index)
                        field_embeddings[item] = df_embeddings.loc[index, 'embedding']
                if field_embeddings:
                    transformed_job_data['embeddings'][key] = field_embeddings

    transformed_job_dict[job_id] = transformed_job_data

    # break

# Print a sample of the transformed dictionary to verify
sample_job_id = next(iter(transformed_job_dict))
print(f"Sample entry for job_id {sample_job_id}:")
for key, value in transformed_job_dict[sample_job_id].items():
    if key == 'embeddings':
        print(f"{key}: {dict(list(value.items())[:2])}...")  # Print only first two items for brevity
    else:
        print(f"{key}: {value}")

# Print the total number of jobs in the transformed dictionary
print(f"\nTotal number of jobs: {len(transformed_job_dict)}")

# Print the number of embedding indexes for the sample job
print(f"\nNumber of embedding indexes for sample job: {len(transformed_job_dict[sample_job_id]['embedding_indexes'])}")



Sample entry for job_id 3940522647:
embeddings: {'company_name': {'AlliedTravelCareers': array([ 0.01218739, -0.04319781,  0.0824301 , ...,  0.00556165,
        0.02249776,  0.00446122])}, 'title': {'Travel PT - $2,410 per week in Plano, TX': array([-0.05116546, -0.00639236, -0.00705088, ..., -0.00220174,
        0.00112664,  0.00669834])}}...
embedding_indexes: [653, 69234, 76282, 76301, 76302, 76453, 77817, 77817, 94757]

Total number of jobs: 99513

Number of embedding indexes for sample job: 9


In [41]:
# Step 2: Add average embeddings
for job_id, job_data in transformed_job_dict.items():
    all_embeddings = []
    for key, value in job_data['embeddings'].items():
        if value == {}:
            continue
        if isinstance(value, dict):
            emb_data = np.stack([emb for emb in value.values()])
            emb_data = np.mean(emb_data, axis=0).tolist()
            all_embeddings.extend([emb_data])
        elif isinstance(value, list):
            all_embeddings.extend([emb for emb in value if emb != []])
    
    all_embeddings = np.stack(all_embeddings, axis=0)
    avg_embedding = np.mean(all_embeddings, axis=0).tolist()
    transformed_job_dict[job_id]['average_embedding'] = avg_embedding
    transformed_job_dict[job_id]['num_embeddings'] = all_embeddings.shape[0]
    # transformed_job_dict[job_id]['embedding_indexes'] = 
    # break

# Print a sample of the transformed dictionary to verify
sample_job_id = next(iter(transformed_job_dict))
print(f"Sample entry for job_id {sample_job_id}:")
print("Embeddings:")
for key, value in transformed_job_dict[sample_job_id]['embeddings'].items():
    print(f"  {key}: {value}")
print("Average Embedding:")
print(transformed_job_dict[sample_job_id]['average_embedding'])  # Printing first 5 elements

# Print the total number of jobs in the transformed dictionary
print(f"\nTotal number of jobs: {len(transformed_job_dict)}")

Sample entry for job_id 3940522647:
Embeddings:
  company_name: {'AlliedTravelCareers': array([ 0.01218739, -0.04319781,  0.0824301 , ...,  0.00556165,
        0.02249776,  0.00446122])}
  title: {'Travel PT - $2,410 per week in Plano, TX': array([-0.05116546, -0.00639236, -0.00705088, ..., -0.00220174,
        0.00112664,  0.00669834])}
  formatted_experience_level: {'Mid-Senior level': array([-0.00968562,  0.01561591,  0.071539  , ..., -0.00657965,
       -0.0031177 ,  0.00574252])}
  job_functions: {'Management': array([-0.03296965,  0.03686457,  0.03200964, ..., -0.03475254,
        0.05776547,  0.00531094]), 'Manufacturing': array([-0.03955336,  0.05302877,  0.02317281, ..., -0.0184946 ,
        0.01590318, -0.01119769])}
  industry_names: {'Hospitals and Health Care': array([-0.04125671, -0.02313279,  0.06126802, ..., -0.01480886,
        0.00963665,  0.02560094])}
  entities_TOOLS: {'BluePipes.': array([-0.01147202,  0.01624093,  0.00757231, ..., -0.07680377,
        0.01882507,

In [42]:
data_ner['average_embedding'] = data_ner['job_id'].apply(lambda x: transformed_job_dict[x]['average_embedding'])
data_ner['num_embedding'] = data_ner['job_id'].apply(lambda x: transformed_job_dict[x]['num_embeddings'])
data_ner['embedding_indexes'] = data_ner['job_id'].apply(lambda x: transformed_job_dict[x]['embedding_indexes'])

In [43]:
data_ner['num_embedding']

0        7
1        7
2        8
3        6
4        8
        ..
99508    8
99509    7
99510    8
99511    6
99512    6
Name: num_embedding, Length: 99513, dtype: int64

In [44]:
data_ner.to_feather('data/data_ner_embeddings.fth')

In [8]:
data_ner = pd.read_feather('data/data_ner_embeddings.fth')

In [18]:
pprint(data_ner.query('wv_uuid == "a4c3e389-a64d-4bf2-b23f-6be8f854d42b"').annotations.values)

array([array([{'end': 14, 'label': 'TITLE', 'start': 0, 'text': 'Data Scientist'},
              {'end': 21, 'label': 'COMPANY', 'start': 17, 'text': 'Dice'},
              {'end': 28, 'label': 'COMPANY', 'start': 24, 'text': 'Dice'},
              {'end': 607, 'label': 'RESPONSABILITY', 'start': 588, 'text': 'build data products'},
              {'end': 887, 'label': 'RESPONSABILITY', 'start': 845, 'text': 'passion for machine-learning and research.'},
              {'end': 1126, 'label': 'RESPONSABILITY', 'start': 998, 'text': 'Provide analytical support and recommendations regarding optimization of operational processes, capital allocation, and pricing.'},
              {'end': 1431, 'label': 'RESPONSABILITY', 'start': 1130, 'text': 'Provide insights into future growth and profitability strategy through historical business analysis and forecasting of industry and major customer trends (e.g. what markets to prioritize, opportunities to pursue, how to win, impact of 5G, etc.) Work wit

In [45]:
df_embeddings.iloc[ 6314]

type                                              company_name
value                                                    Jobot
embedding    [-0.012203977443277836, -0.03190867230296135, ...
Name: 6314, dtype: object

In [26]:
df_embeddings.iloc[76279]

type                                formatted_experience_level
value                                              Entry level
embedding    [-0.011425187811255455, 0.006456522736698389, ...
Name: 76279, dtype: object

In [23]:
len(transformed_job_dict.keys())

99513

In [14]:
len(data_ner)

99513

In [15]:
data_ner.columns

Index(['job_id', 'scraped', 'company_id', 'work_type', 'formatted_work_type',
       'location', 'job_posting_url', 'applies', 'original_listed_time',
       'remote_allowed', 'application_url', 'application_type', 'expiry',
       'inferred_benefits', 'closed_time', 'formatted_experience_level',
       'years_experience', 'description', 'title', 'skills_desc', 'views',
       'job_region', 'listed_time', 'degree', 'posting_domain', 'sponsored',
       'country', 'country_code', 'job_functions', 'industry_names',
       'company_name', 'description_company', 'company_size', 'state',
       'country_company', 'city', 'zip_code', 'address', 'url', 'text',
       'entities_COMPANY', 'entities_METHODS', 'entities_TOOLS',
       'entities_EXPERIENCE', 'entities_LEVEL', 'entities_REMOTE',
       'entities_RESPONSABILITY', 'entities_TITLE', 'entities_QUALIFICATION',
       'vector', 'wv_uuid', 'annotations', 'average_embedding',
       'num_embedding', 'embedding_indexes'],
      dtype='objec

In [17]:
for i,j in df_embeddings.iterrows():
    print(i)
    print(j)
    break

0
type                                              company_name
value                        Coeur d’Alene Casino Resort Hotel
embedding    [0.018065080046653748, -0.0579984188079834, 0....
Name: 0, dtype: object


In [19]:
from typing import List
import os
import openai
from main.keys import open_ai_key, weaviate_url, weaviate_key
import os
import weaviate
import openai

os.environ["OPENAI_APIKEY"] = open_ai_key
os.environ["WCD_URL"] = weaviate_url
os.environ["WCD_API_KEY"] = weaviate_key

openai_api_key = os.environ.get("OPENAI_APIKEY", "<your OpenAI API key if not set as env var>")
openai.api_key = openai_api_key
# Define a function to call the endpoint and obtain embeddings
def vectorize(texts: List[str]) -> List[List[float]]:

    response = openai.embeddings.create(
        input=texts, model="text-embedding-3-small"
    )

    return response.data[0].embedding

x_text = "Machine Learning Engineer"
x_vector = vectorize([x_text])

y_text = "Data Scientist"
y_vector = vectorize([y_text])

z_text = "Accountant"
z_vector = vectorize([z_text])

In [20]:
def cosine_distance( a, b):
    #Cosine distance is 1 - cosine similarity
    # cos(theta) = (a . b) / (||a|| ||b||)
    cos_sim = np.dot(a, b)/(np.linalg.norm(a)*(np.linalg.norm(b)))
    return 1 - cos_sim

In [21]:
df = data_ner.copy()

In [ ]:
import pandas as pd
import plotly.graph_objects as go
from plotly.offline import plot
# Assuming df is your DataFrame and it has columns 'tsne-2d-one', 'tsne-2d-two' and 'title'
df = df.copy()
df['x_dist'] = df['average_embedding'].apply(lambda x: cosine_distance(x, x_vector))
df['y_dist'] = df['average_embedding'].apply(lambda x: cosine_distance(x, y_vector))
df['z_dist'] = df['average_embedding'].apply(lambda x: cosine_distance(x, z_vector))

distance_df = df.copy()
x = distance_df['x_dist'].values
y = distance_df['y_dist'].values
z = distance_df['z_dist'].values
num_embeddings = distance_df['num_embedding'].values
global_min = min(x.min(), y.min(), z.min())
global_max = max(x.max(), y.max(), z.max())
fig = go.Figure(data=go.Scatter3d(
    x=x,
    y=y,
    z=z,
    mode='markers',
    marker=dict(
        size=8,
        color=x+y+z ,  # set color to cluster values
        colorscale='Viridis',  # choose a colorscale
        opacity=0.8,
        colorbar=dict(title='Semantic Distance')
    ),
    # marker=dict(size=8, opacity=0.5),
    text=distance_df['title']+' | '+distance_df['company_name']+' | '+distance_df['num_embedding'].astype(str),  # this will set the hover text
    hoverinfo='text',
    name='Semantic Distance'

    
))

fig.update_layout(title='t-SNE plot',
                  scene=dict(
                      xaxis_title=x_text,
                      yaxis_title=y_text,
                      zaxis_title=z_text,
                      aspectmode='cube',
                      xaxis=dict(range=[global_min, global_max]),  # set range for x axis
                      yaxis=dict(range=[global_min, global_max]),  # set range for y axis
                      zaxis=dict(range=[global_min, global_max]),  # set range for z axis
                  ),
                  hovermode='closest')


fig.show()

plot(fig)

In [ ]:
import pandas as pd
import plotly.graph_objects as go
from plotly.offline import plot
# Assuming df is your DataFrame and it has columns 'tsne-2d-one', 'tsne-2d-two' and 'title'
df = df.copy()
df['x_dist'] = df['vector'].apply(lambda x: cosine_distance(x, x_vector))
df['y_dist'] = df['vector'].apply(lambda x: cosine_distance(x, y_vector))
df['z_dist'] = df['vector'].apply(lambda x: cosine_distance(x, z_vector))

distance_df = df.copy()
x = distance_df['x_dist'].values
y = distance_df['y_dist'].values
z = distance_df['z_dist'].values
num_embeddings = distance_df['num_embedding'].values
global_min = min(x.min(), y.min(), z.min())
global_max = max(x.max(), y.max(), z.max())
fig = go.Figure(data=go.Scatter3d(
    x=x,
    y=y,
    z=z,
    mode='markers',
    marker=dict(
        size=8,
        color=x+y+z ,  # set color to cluster values
        colorscale='Viridis',  # choose a colorscale
        opacity=0.8,
        colorbar=dict(title='Semantic Distance')
    ),
    # marker=dict(size=8, opacity=0.5),
    text=distance_df['title']+' | '+distance_df['company_name']+' | '+distance_df['num_embedding'].astype(str),  # this will set the hover text
    hoverinfo='text',
    name='Semantic Distance'

    
))

fig.update_layout(title='t-SNE plot',
                  scene=dict(
                      xaxis_title=x_text,
                      yaxis_title=y_text,
                      zaxis_title=z_text,
                      aspectmode='cube',
                      xaxis=dict(range=[global_min, global_max]),  # set range for x axis
                      yaxis=dict(range=[global_min, global_max]),  # set range for y axis
                      zaxis=dict(range=[global_min, global_max]),  # set range for z axis
                  ),
                  hovermode='closest')


fig.show()

plot(fig)

'temp-plot.html'

In [3]:
client = weaviate.connect_to_local(host='45.79.238.197',auth_credentials=weaviate.auth.AuthApiKey(weaviate_key))
# client.collections.delete("ListingsV3_SCE")

In [1]:
listings_wvc = client.collections.create(
    name="ListingsV3_SCE",
    vectorizer_config=wvc.config.Configure.Vectorizer.text2vec_openai(model='text-embedding-3-small'),    # Set the vectorizer to "text2vec-openai" to use the OpenAI API for vector-related operations
    generative_config=wvc.config.Configure.Generative.openai(),             # Set the generative module to "generative-cohere" to use the Cohere API for RAG
    properties = [
        wvc.config.Property(
            name="job_id",
            data_type=wvc.config.DataType.INT,
            description="Unique identifier for the job posting",
            skip_vectorization=True
        ),
        wvc.config.Property(
            name="company_id",
            data_type=wvc.config.DataType.INT,
            description="Unique identifier for the company",
            skip_vectorization=True
        ),
        wvc.config.Property(
            name="title",
            data_type=wvc.config.DataType.TEXT,
            description="Job Listing Title",
        ),
        wvc.config.Property(
            name="company_name",
            data_type=wvc.config.DataType.TEXT,
            description="Name of the company",
        ),
        wvc.config.Property(
            name="formatted_experience_level",
            data_type=wvc.config.DataType.TEXT,
            description="Formatted experience level for the job",
             
        ),
        wvc.config.Property(
            name="job_functions",
            data_type=wvc.config.DataType.TEXT_ARRAY,
            description="List of job functions",
             
        ),
        wvc.config.Property(
            name="industry_names",
            data_type=wvc.config.DataType.TEXT_ARRAY,
            description="List of industry names",
             
        ),
        wvc.config.Property(
            name="entities_TOOLS",
            data_type=wvc.config.DataType.TEXT_ARRAY,
            description="List of tools mentioned in the job posting",
             
        ),
        wvc.config.Property(
            name="entities_METHODS",
            data_type=wvc.config.DataType.TEXT_ARRAY,
            description="List of methods mentioned in the job posting",
             
        ),
        wvc.config.Property(
            name="entities_REMOTE",
            data_type=wvc.config.DataType.TEXT_ARRAY,
            description="List of remote work related entities",
             
        ),
        wvc.config.Property(
            name="entities_EXPERIENCE",
            data_type=wvc.config.DataType.TEXT_ARRAY,
            description="List of experience mentioned in the job posting",
             
        ),
        wvc.config.Property(
            name="entities_RESPONSABILITY",
            data_type=wvc.config.DataType.TEXT_ARRAY,
            description="List of responsibility mentioned in the job posting",
             
        ),
        wvc.config.Property(
            name="description",
            data_type=wvc.config.DataType.TEXT,
            description="Original job description text",
            skip_vectorization=True
        ),
    ]
)


NameError: name 'client' is not defined

_CollectionConfig(name='ListingsV3_SCE',
                  description=None,
                  generative_config=_GenerativeConfig(generative=<GenerativeSearches.OPENAI: 'generative-openai'>,
                                                      model={}),
                  inverted_index_config=_InvertedIndexConfig(bm25=_BM25Config(b=0.75,
                                                                              k1=1.2),
                                                             cleanup_interval_seconds=60,
                                                             index_null_state=False,
                                                             index_property_length=False,
                                                             index_timestamps=False,
                                                             stopwords=_StopwordsConfig(preset=<StopwordsPreset.EN: 'en'>,
                                                                                        additions

In [10]:
weaviate_load = True
data_ner = pd.read_feather('data/data_ner_embeddings.fth')
data = data_ner.copy()

data['entities_TOOLS'] = data['entities_TOOLS'].apply(lambda x: x.tolist())
data['entities_METHODS'] = data['entities_METHODS'].apply(lambda x: x.tolist())
data['entities_REMOTE'] = data['entities_REMOTE'].apply(lambda x: x.tolist())
data['entities_EXPERIENCE'] = data['entities_EXPERIENCE'].apply(lambda x: x.tolist())
data['entities_RESPONSABILITY'] = data['entities_RESPONSABILITY'].apply(lambda x: x.tolist())
data['job_functions'] = data['job_functions'].apply(lambda x: x.tolist())
data['industry_names'] = data['industry_names'].apply(lambda x: x.tolist())
if weaviate_load:

    listings_objs = data[['job_id','company_id','company_name','title','formatted_experience_level','job_functions', 'industry_names', 'entities_TOOLS', 'entities_METHODS', 'entities_REMOTE', 'entities_EXPERIENCE', 'entities_RESPONSABILITY', 'description', ]].fillna('').to_dict(orient="records")
    vectors = data['average_embedding']
    pprint(listings_objs[0])


    listings = client.collections.get("ListingsV3_SCE")

    listings_wvc_obs = [wvc.data.DataObject(properties=x, vector=v) for x,v in zip(listings_objs,vectors)]
    # listings_list= list()
    for i in range(0,len(listings_wvc_obs),1000):
        print(i)
        # try:
        listings.data.insert_many(listings_wvc_obs[i:i+1000])
        # listings_list.append()
        # break

{'company_id': 11092369.0,
 'company_name': 'AlliedTravelCareers',
 'description': 'AlliedTravelCareers is working with FlexCare Medical Staffing '
                'to find a qualified PT in Plano, Texas, 75023! Pay '
                'Information $2,410 per week About The Position FlexCare is a '
                'nationwide leader in the staffing of travel nurses and '
                'clinicians. With access to thousands of facilities around the '
                'country, our mission is to deliver premier travel staffing '
                'solutions with integrity and transparency. FlexCare’s '
                'exceptional service and reputation make it one of the most '
                'awarded healthcare staffing companies in the industry each '
                'year, ranking as one of the top travel nursing companies in '
                'the nation from industry authorities like Highway '
                'Hypodermics, Travel Nursing Central, and BluePipes. FlexCare '
            

In [11]:
item_details = {}
test = ''
for item in listings.iterator(include_vector=True):
    item_details[item.properties['job_id']] = {'uuid': item.uuid, 'embedding': item.vector['default']}

In [6]:
listings.__properties

AttributeError: 'Collection' object has no attribute '__properties'

In [12]:
data_ner['wv_uuid'] = data_ner['job_id'].apply(lambda x: str(item_details[x]['uuid']))

In [24]:
def generate_axis_summary_prompt(text):
    prompt = f"""These job listings are the most aligned with the "{text}" axis based on their cosine distances.

Please provide a concise summary that highlights the key aspects of these job listings 
that contribute to their alignment with the "{text}" axis. Consider the following points in your summary:

1. What are the common themes, skills, or requirements mentioned across these job listings?
2. How do these job listings relate to the concept of "{text}"?
3. Are there any specific qualifications, experience, or responsibilities that make these job listings highly relevant to "{text}"?

Please format your response as follows:

Summary: [A concise summary of the key aspects that make these job listings aligned with the "{text}" axis]

Common Themes:
- [Theme 1]
- [Theme 2]
- [Theme 3]

Relevance to "{text}":
- [Point 1]
- [Point 2]
- [Point 3]

Specific Qualifications or Experience:
- [Qualification 1]
- [Qualification 2]
- [Qualification 3]"""
    
    return prompt

In [25]:
listings = client.collections.get("ListingsV3_SCE")

In [28]:
properties = ['job_id','company_id','company_name','title','formatted_experience_level','job_functions', 'industry_names', 'entities_TOOLS', 'entities_METHODS', 'entities_REMOTE', 'entities_EXPERIENCE', 'entities_RESPONSABILITY',]
prompt = generate_axis_summary_prompt(['Machine Learning Engineer'])
response = listings.generate.near_text(query=['Data Scientist'], grouped_task=prompt, limit=20, grouped_properties=properties).generated

In [29]:
pprint(response)

('Summary: The job listings are aligned with the "[\'Machine Learning '
 'Engineer\']" axis as they primarily focus on roles related to data science, '
 'machine learning, and predictive analytics. They require strong technical '
 'skills, experience with programming languages like Python and R, and the '
 'ability to develop and implement machine learning models.\n'
 '\n'
 'Common Themes:\n'
 '- Data science and machine learning\n'
 '- Strong technical skills in Python and R\n'
 '- Developing and implementing machine learning models\n'
 '\n'
 'Relevance to "[\'Machine Learning Engineer\']":\n'
 '- These job listings are highly relevant to the concept of a "Machine '
 'Learning Engineer" as they involve working with data, developing machine '
 'learning models, and utilizing predictive analytics to solve complex '
 'problems.\n'
 '- The roles require a deep understanding of machine learning algorithms, '
 'statistical modeling, and data analysis techniques.\n'
 '- The job listings emph

In [ ]:
pprint(listings.config.get())

In [ ]:
listings.

In [14]:
data_ner.to_feather('data/data_ner_embeddings.fth')